In [1]:
execution_mode = 'restricted'
factor = 0.1
exactDate_mode = 'added_u'

# Feature Matrix Generation

In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), Swissbib's goldstandard data has been processed to form records of pairs of duplicate and pairs of unique records. These records are the starting point for the final feature matrix generation and that is the reason, why the DataFrame was called feature base. As described in [[JudACaps](./A_References.ipynb#judacaps)], the next step will be an attribute-wise comparison of each attribute pair of each record in the original feature base. This comparison will generate similarity values for each attribute pair. It will halve the number of attributes replacing each attribute pair with one value expressing their degree of similarity. The goal of this chapter is a DataFrame with the full and final feature attributes. The values of these feature attributes will be used for training and performance testing of the machine learning models in the chapters to follow.

This chapter introduces similarity metrics for string comparisons. The metrics to be used for calculating its similarity will be decided for each attribute pair of the DataFrame built in the previous chapters.

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Object Distance and Similarity](#Object-Distance-and-Similarity)
    - [Mathematical Definitions](#Mathematical-Definitions)
    - [Library TextDistance](#Library-TextDistance)
- [Similarity Metrics on Attribute Level](#Similarity-Metrics-on-Attribute-Level)
    - [Table of Contents of Attribute Similarities](#Table-of-Contents-of-Attribute-Similarities)
- [DataFrame with Attributes and Similarity Features](#DataFrame-with-Attributes-and-Similarity-Features)
- [Summary](#Summary)
    - [Full Feature Matrix with Target Vector Handover](#Full-Feature-Matrix-with-Target-Vector-Handover)

## Data Takeover

Swissbib's raw data of the goldstandard has been processed in chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb). As the first step of this chapter, this data is loaded for further processing to the feature matrix and target vector for the subsequent machine learning model chapters.

In [2]:
import os
import pandas as pd
import pickle as pk
import bz2
import _pickle as cPickle

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore DataFrame with features from compressed pickle file
with bz2.BZ2File((os.path.join(
    path_goldstandard, 'feature_base_df.pkl')), 'rb') as file:
    df_feature_base = cPickle.load(file)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,035liste_x,035liste_y,century_x,century_y,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,coordinate_x,coordinate_y,corporate_110_x,corporate_110_y,corporate_710_x,corporate_710_y,corporate_full_x,corporate_full_y,decade_x,decade_y,docid_x,docid_y,doi_x,doi_y,duplicates,edition_x,edition_y,exactDate_x,exactDate_y,format_postfix_x,format_postfix_y,format_prefix_x,format_prefix_y,isbn_x,isbn_y,ismn_x,ismn_y,masters_docid,musicid_x,musicid_y,pages_x,pages_y,part_x,part_y,person_100_x,person_100_y,person_245c_x,person_245c_y,person_700_x,person_700_y,pubinit_x,pubinit_y,pubword_x,pubword_y,pubyear_x,pubyear_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,ttlpart_x,ttlpart_y,volumes_x,volumes_y
0,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,"grawechristian, graweursula","grawechristian, graweursula",reclam jun.,reclam jun.,[Reclam jun.],[Reclam jun.],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600 s.,600 s.
1,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman",emma,,,"{'245': ['Emma', 'Roman']}",{'245': ['Emma']},600 s.,600 s.
2,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (LIBIB)000315536]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,323173349,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600 s.,600 s.
3,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,,"grawechristian, graweursula",reclam,reclam jun.,[Reclam],[Reclam jun.],2009,2009,,,emma,"emma, roman",,,{'245': ['Emma']},"{'245': ['Emma', 'Roman']}",600 s.,600 s.
4,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,,,reclam,reclam,[Reclam],[Reclam],2009,2009,,,emma,emma,,,{'245': ['Emma']},{'245': ['Emma']},600 s.,600 s.


Chapter [Overview and Summary](./0_OverviewSummary.ipynb) assesses the predictions of the various models. One measure to analyse the results will be the resulting confusion matrix for a model prediction. The confusion matrix will reveal cases in the testing data that the model will predict opposite to the target values of the testing data. These predictions are called false positives and false negatives. To be able to analyse the original attribute values of the affected records in detail, columns $\texttt{035liste}\_\texttt{x}$, $\texttt{035liste}\_\texttt{y}$, $\texttt{docid}\_\texttt{x}$, and $\texttt{docid}\_\texttt{y}$ need to be restored. These four attributes will now be saved to a separate DataFrame to be reloaded in later chapters.

In [3]:
# Store docid's for fast identification of row pairs in model results
df_index_docids = df_feature_base[['035liste_x', '035liste_y', 'docid_x', 'docid_y']]
# The DataFrame of pairs with target information
df_feature_base = df_feature_base[columns_metadata_dict['columns_to_use']]

df_feature_base.sample(n=5)

,duplicates,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,corporate_full_x,corporate_full_y,doi_x,doi_y,edition_x,edition_y,exactDate_x,exactDate_y,format_prefix_x,format_prefix_y,format_postfix_x,format_postfix_y,isbn_x,isbn_y,ismn_x,ismn_y,musicid_x,musicid_y,part_x,part_y,person_100_x,person_100_y,person_700_x,person_700_y,person_245c_x,person_245c_y,pubinit_x,pubinit_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,volumes_x,volumes_y
29071,0,,,,,,,,,5,,2000uuuu,1994uuuu,bk,bk,020000,020000,[3-254-08008-4],[],,,,,bd. 8008,1,mozartwolfgang amadeus,,,,wolfgang amadeus mozart ; einf. und kommentar ...,[sigrid kessler... et al.] ; [hrsg.] interkant...,,[staatlicher lehrmittelverlag],,,"die zauberflöte, textbuch","bonne chance!, cours de langue française, 1, d...",,,270 s.,93 s.
56698,0,,,,,,,,,,,2012uuuu,1793uuuu,bk,bk,020300,020000,[978-3-7255-6535-1],[],,,,,"63, 63",,käserbeatrice,mozartwolfgang amadeus,,schikanederemanuel,beatrice käser,von emmanuel schikaneder ; die musik ist von h...,schulthess,[s.n.],,,"sozialleistungsbetrug, sozialversicherungsbetr...","die zauberflöte, eine grosse oper in zwey aufz...",,,"lx, 244 s.",96 s.
61011,0,,,,,,,,,,,1909uuuu,19961997,bk,vm,020000,030000,[],[],,,,,,11,mozartwolfgang amadeus,,"schikanederemanuel, loewenfeldhans, leflerhein...",,textbuch von emanuel schikaneder ; szenische e...,sigrid kessler... [et al.] ; [éd.:] interkanto...,,"interkantonale lehrmittelzentrale, berner lehr...",,,die zauberflöte,"bonne chance!, cours de langue française, deux...",,,1 textbuch,6 disques compacts
91123,0,,,,,,,,,,,1998uuuu,1797uuuu,bk,mu,020047,010000,[3-932992-42-3],[],,,,542,,,frischmax,mozartwolfgang amadeus,,,max frisch,composé par w.a. mozart ; arrangé pour le clav...,terzio,,,,homo faber,"die zauberflöte, grand opera",,,1 compact disc/cd-rom [windows],stimmen
7373,0,,,,,,,,,,,2012uuuu,2008uuuu,bk,bk,020300,020053,[978-3-7255-6535-1],"[978-3-598-31797-2 (print), 978-3-11-092367-4]",,,,,bd. 63,"bang 50, bang 50",käserbeatrice,mortzfeldpeter,,raabepaul,beatrice käser,"mortzfeld, peter; raabe, paul",schulthess,de gruyter saur,,,sozialleistungsbetrug - sozialversicherungsbet...,katalog der graphischen porträts in der herzog...,,,244 s.,1 online resource (383 p.)


In [4]:
print('Number of rows labelled as duplicates {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==1])))
print('Number of rows labelled as uniques {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==0])))
print('Total number of rows in DataFrame {:,d}'.format(df_feature_base.shape[0],
      'number of columns', df_feature_base.shape[1]))

Number of rows labelled as duplicates 1,473
Number of rows labelled as uniques 257,955
Total number of rows in DataFrame 259,428


In [5]:
print('Part of duplicates (1) on uniques (2) in units of [%]')
print(round(100*df_feature_base.duplicates.value_counts(normalize=True), 1))

Part of duplicates (1) on uniques (2) in units of [%]
0    99.4
1     0.6
Name: duplicates, dtype: float64


DataFrame feature base is the starting point used for the further processing in this chapter.

## Object Distance and Similarity

A mathematical idea of distance and similarity is needed for understanding object pair comparison. This section starts with a motivation for calculating similarities and afterwards gives a very basic definition of the two central terms, distance and similarity. The text of this section is a summary of [[Chri2012](./A_References.ipynb#chri2012)].

### Mathematical Definitions

The attributes to be used for pair comparison may contain values of poor quality. The quality originates in the way the data has been entered at the very source. Manual data entry may suffer from mistyping, automatically scanned data may suffer from insufficiencies of the scanned base material or the recognition algorithm in the optical character recognition (OCR) processing. The basic step of a deduplication process is to identify the probability of two strings of a pair to be a pair of duplicates. This is done by calculating a similarity value between the two strings compared, rather than using an exact comparison function. Based on this common similarity value for an attribute pair, their being duplicates can be decided.

The term similarity is strongly coupled to the term of distance of two values of an attribute. Mathematically, a distance can be explained with the help of a distance function. A _distance function_ or _distance metric_ $dist(o_i, o_j)$ between two points or data objects $o_i$ and $o_j$ must fulfill four requirements.

1. $dist(o_i, o_i)=0$, the distance from an object to itself is zero.
- $dist(o_i, o_j)\ge 0$, the distance between two objects is a non-negative number.
- $dist(o_i, o_j)=dist(o_j, o_i)$, the distance between two objects is symmetric.
- $dist(o_i, o_j)\le dist(o_i, o_k)+dist(o_k, o_j)$, the triangular inequality must hold. It states that the direct distance beween two objects is never larger than the combined distance when going through a third object.

A distance value expresses the dissimilarity $d$ of two objects [[HanK2012](./A_References.ipynb#hank2012)] and can therefore be converted into a similarity value $s$, calculating $s = \frac{1}{d}$, assuming $d\gt 0$. Alternatively, assuming the distance value is normalised $0\le d\le 1$, the similarity value can be calculated to $s = 1-d$. A _similarity function_ $sim(a_i, aj)$ between two attributes which can be strings, numbers, dates, geographic locations, text, XML documents, etc. fulfills the general requirements.

1. $sim(a_i, a_i)=1$, the result of comparing a value with itself is an exact similarity.
- $sim(a_i, a_j)=0$, the similarity of values that are completely different from each other is 0. What accounts for 'complete different' depends upon the type of data that are compared.
- $0\lt sim(a_i, a_j)\lt 1$, an approximate similarity between exact similarity and total dissimilarity is calculated if two attribute values are somewhat similar to each other. What accounts for 'somewhat different' depends upon the type of data that are compared.

The dissimilarity between two objects $o_i$ and $o_j$ can be computed based on the ratio of mismatches,
$$
d(o_i, o_j) = \frac{p-m}{p},
$$
where $m$ is the number of matching attributes and $p$ is the total number of attributes describing the objects [[HanK2012](./A_References.ipynb#hank2012)]. Thus the similarity between two objects can be computed as
$$
sim(o_i, o_j) = 1 - d(o_i, o_j) = \frac{m}{p}.
$$

For data deduplication, a comparison function needs to be tailored to the type of underlying data. Although there is a correspondence between a similarity function and the mathematical concept of a distance function, not all known and implemented similarity comparison functions used for string pair comparison fulfill the requirements of a distance function. Some similarity functions are not symmetric, others do not fulfill the triangular inequality. Decision taking on the best similarity function for a string pair, will be based on the effect, a similarity function has for the purpose needed. In the case of this capstone project, this purpose is its capability to contribute to the prediction whether a pair of records is a pair duplicates or a pair of uniques.

### Library TextDistance

An internet research on string distance calculation with Python has revealed libraries [[StSi](./A_References.ipynb#stsi)], [[TeDi](./A_References.ipynb#tedi)] and seperate code snippets for distinct algorithms. After trying the referenced libraries and a downloaded code snippet for a Smith Waterman similarity [[SmWa](./A_References.ipynb#smwa)], the text distance library [[TeDi](./A_References.ipynb#tedi)] has been decided to be the best decision for this capstone project. The decision is based on the GitHub statistics of stars and the date of the latest pull requests, indicating its popularity and maintenance activity of the library. A look at the API of the library, reveals the Python library to be a complete implementation (compared to suggestions of similarity metrics in [[Chri2012](./A_References.ipynb#chri2012)]) and easy to use.

In [6]:
# Install textdistance Python library - if not done, yet.
! pip install textdistance

For using the library, see documentation in [[TeDi](./A_References.ipynb#tedi)]. For the purposes of this chapter, function $\texttt{.normalized}\_\texttt{similarity()}$ of an instantiated textdistance object will be used.

In [7]:
import textdistance as tedi

With the code line above, the library is imported for application in this chapter. In appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb) the effects of the similarity metrics of the library are compared for better understanding of their specific behaviour. This comparison for each attribute is the basis of deciding the best similarity metric available for an attribute pair.

## Similarity Metrics on Attribute Level

This section implements the decision for calculating the similarity metric for each attribute of the raw data based on appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The implementation is applied on a pair of attributes of two records, resulting in a new attribute, the similarity value, of the final feature matrix. A general function $\texttt{build_delta_feature}$ is provided by the code file [data_preparation_funcs.py](./data_preparation_funcs.py) for transforming two attributes into their feature attribute holding their similarity value.

In [8]:
import data_preparation_funcs as dpf

### Table of Contents of Attribute Similarities

- [coordinate](#coordinate)
- [corporate](#corporate)
- [doi](#doi)
- [edition](#edition)
- [exactDate](#exactDate)
- [format](#format)
- [isbn](#isbn)
- [ismn](#ismn)
- [musicid](#musicid)
- [part](#part)
- [person](#person)
- [pubinit](#pubinit)
- [scale](#scale)
- [ttlfull](#ttlfull)
- [volumes](#volumes)

In [9]:
# These dictionary attributes will be filled in function dpf.build_delta_feature()
columns_metadata_dict['similarity_metrics'] = {}
columns_metadata_dict['features'] = []

### coordinate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{coordinate}$ holds coordinates of maps. To decide whether a map covers the same geographical range, a metric will be chosen that compares the coordinate number digits from left to right. The more digits are found to be the equal, the higher the similarity value is calculated. The comparison stops with the first digit pair that differs. This algorithm is satisfyed by the LCS (Longest Common Substring comparison) algorithm and generates the wanted result, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb).

In [10]:
attribute = 'coordinate'

columns_metadata_dict['similarity_metrics'][attribute+'_E'] = tedi.LCSStr()
columns_metadata_dict['similarity_metrics'][attribute+'_N'] = tedi.LCSStr()

ne_values = ['_E', '_N']

for ne in ne_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+ne,
        columns_metadata_dict['similarity_metrics'][attribute+ne],
        columns_metadata_dict)

The length of attribute $\texttt{coordinate}$ is exactly eight digits. The distinct similarity values that may occurr form a discrete set of values with a distance of $\frac{1}{8}$ between adjacent values.

In [11]:
uniques, uniques_len = {}, {}

for ne in ne_values :
    uniques[attribute+ne], uniques_len[attribute+ne] = dpf.determine_similarity_values(
        df_feature_base, attribute+ne)

coordinate_E values range [0.    0.375 0.5   0.625 0.875 1.   ]
coordinate_N values range [0.    0.375 0.5   0.75  0.875 1.   ]


Looking at some samples of the feature matrix reveals a good match to the expectations.

In [12]:
position = 3

for ne in ne_values :
    dpf.show_samples_interval(
        df_feature_base, attribute+ne,
        uniques[attribute+ne][uniques_len[attribute+ne]-position],
        uniques[attribute+ne][uniques_len[attribute+ne]-position+1]
    )

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
69543,0,0.875,e0080855,e0080851
96189,0,0.875,e0080851,e0080855
96184,0,0.875,e0080851,e0080855
865,1,0.875,e0080851,e0080855
196658,0,0.625,e0055700,e0055009


0.625 <= coordinate_E_delta <= 0.875


,duplicates,coordinate_N_delta,coordinate_N_x,coordinate_N_y
96173,0,0.750,n0460821,n0460833
182754,0,0.750,n0460826,n0460833
96192,0,0.750,n0460821,n0460833
55311,0,0.875,n0460826,n0460821
182752,0,0.750,n0460826,n0460833


0.75 <= coordinate_N_delta <= 0.875


The samples above show the wanted similarity behaviour for value ranges greater than 0. The metric has the weakness, though, that empty coordinate values, e.g. for bibliographical units other than maps, have each been calculated to a similarity of 0. Some samples for duplicates in the training data are shown below.

In [13]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute+'_E', uniques[attribute+'_E'][0], uniques[attribute+'_E'][1], 10)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
389,1,0.0,,
336,1,0.0,,
1296,1,0.0,,
1318,1,0.0,,
1050,1,0.0,,
24,1,0.0,,
1305,1,0.0,,
203,1,0.0,,
158,1,0.0,,
687,1,0.0,,


0.0 <= coordinate_E_delta <= 0.375


This downside shall be avoided by marking pairs of missing coordinate values on both sides with a special negative value, which will point out to the models to be trained, the special case of missing information in a row. The implementation of this logic is done in function $\texttt{.mark}\_\texttt{missing}()$. The absolute value of this negative number is conrolled by a factor which is passed to the function as a parameter. The function handles explicitly two cases. The first one is missing information in both attributes (resulting in $-1*\texttt{factor}$) and the second one is missing information in only one of the two attributes (resulting in $-0.5*\texttt{factor}$).

In [14]:
for ne in ne_values :
    df_feature_base = dpf.mark_missing(df_feature_base, attribute+ne, factor)

### corporate

Attribute $\texttt{corporate}$ is a collection of corporate names. The Monge-Elkan metric compares string tokens pairwise [[Chri2012](./A_References.ipynb#chri2012)] while the LCS metric searches for the longest common substring. Assessing the differences of these two metrics with the help of their values distribution in chapter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb), reveals a better distribution behaviour for LCS. Therefore, the LCS metric will be chosen for this attribute.

In [15]:
attribute = 'corporate_full'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.StrCmp95()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [16]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

corporate_full values range [0.         0.01333333 0.01428571 0.01639344 0.01666667 0.01694915
 0.01818182 0.01960784 0.02       0.02040816 0.0212766  0.02173913
 0.02272727 0.02380952 0.025      0.02531646 0.02542373 0.02666667
 0.02702703 0.02727273 0.02857143 0.03       0.03030303 0.03125
 0.03225806 0.03278689 0.03333333 0.03389831 0.03448276 0.03571429
 0.03636364 0.03773585 0.03797468 0.03846154 0.03921569 0.04
 0.04081633 0.04166667 0.04237288 0.04255319 0.04285714 0.04347826
 0.04444444 0.04545455 0.046875   0.04761905 0.04878049 0.04918033
 0.05       0.05263158 0.05333333 0.05405405 0.05454545 0.05555556
 0.05660377 0.05714286 0.05882353 0.05932203 0.06       0.06060606
 0.06122449 0.0625     0.06329114 0.06363636 0.06382979 0.06451613
 0.06521739 0.06557377 0.06666667 0.06779661 0.06818182 0.06896552
 0.07       0.07142857 0.07272727 0.07317073 0.075      0.0754717
 0.07594937 0.07627119 0.07692308 0.078125   0.07843137 0.07894737
 0.08       0.08108108 0.08163265 0.08181818

Its $110$ part is sparsely filled and even its $710$ part comes along with a little more than $10\%$ of filling, only. The LCS metric generates a similarity of 1 for the cases where both strings of a pair are empty. Missing values on both sides may be an indicator for a pair of duplicates but due to the sparsely available information, it is a weak indicator. Therefore, the pairs with missing data on both sides of the pair, will be marked with the negative value.

In [17]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

Some sample cases are shown below for both $\texttt{corporate}$ features.

In [18]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute, 0.0, 1.0, 20
)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
221,1,0.49,schweizerische gesellschaft für bildungsforsch...,schweizerische gesellschaft für bildungsforschung
187,1,0.49,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforsch...
550,1,1.00,eidgenössisches topographisches bureau eidgenö...,eidgenössisches topographisches bureau eidgenö...
191,1,1.00,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforschung
188,1,0.49,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforsch...
1414,1,1.00,schweizerische normen-vereinigung,schweizerische normen-vereinigung
757,1,1.00,société suisse pour la recherche en éducation,société suisse pour la recherche en éducation
182,1,1.00,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforschung
1260,1,1.00,springerlink (online service),springerlink (online service)
1309,1,1.00,schweizeidgenössisches topographisches bureau,schweizeidgenössisches topographisches bureau


0.0 <= corporate_full_delta <= 1.0


In [19]:
position = uniques_len[attribute]//2 # Let's have a look in the middle range of the similarities.

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 20)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
38328,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
38371,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenössisches topographisches bureau
136349,0,0.101266,"interkantonale lehrmittelzentrale (luzern), st...","rundfunkchor (leipzig), staatskapelle (dresden)"
47819,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
168197,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
25876,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
38332,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
168196,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
86955,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
157170,0,0.101695,allgemeine geschichtforschende gesellschaft de...,"historischer verein des kantons solothurn, kon..."


0.10126582278481011 <= corporate_full_delta <= 0.10204081632653061


### doi

Swissbib uses an explicit $\texttt{doi}$ attribute for its deduplication implementation. In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), the real doi identifier has been isolated with the help of a preprocessing function $\texttt{.reduce}\_\texttt{to}\_\texttt{doi}\_\texttt{element()}$, see [Data Analysis](./1_DataAnalysis.ipynb). Attribute $\texttt{doi}$ contains a single string value. The Identity metric will be used for comparing the string values of a pair in a row, calculating a similarity value of 1.0 or 0.0 for each pair. If one list is empty a value of 0 is returned.

In [20]:
attribute = 'doi'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base['doi_delta'].unique()

array([1., 0.])

Some sample cases are shown below for each category of $\texttt{doi}\_\texttt{delta}$.

In [21]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

doi values range [0. 1.]


,duplicates,doi_delta,doi_x,doi_y
176009,0,1.0,,
123874,0,1.0,,
204036,0,1.0,,
79506,0,1.0,,
141377,0,1.0,,
51124,0,1.0,,
233559,0,1.0,,
239690,0,1.0,,
104078,0,1.0,,
21748,0,1.0,,


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
124001,0,0.0,,10.5169/seals-377028
132113,0,0.0,,10.3931/e-rara-61897
207448,0,0.0,10.1093/cid/ciu795,
101501,0,0.0,,10.5169/seals-515343
153654,0,0.0,,10.5169/seals-377160
126179,0,0.0,,10.3931/e-rara-60823
201094,0,0.0,,10.5169/seals-377160
115113,0,0.0,,10.5169/seals-377277
167979,0,0.0,10.1093/cid/ciu795,
152496,0,0.0,,10.5169/seals-515306


doi_delta = 0.0


In [22]:
# Let's have a look at some non-empty doi elements
df_doi_with_element = df_feature_base[df_feature_base.doi_x.apply(lambda x : len(x) > 0)]

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_doi_with_element, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

,duplicates,doi_delta,doi_x,doi_y
1262,1,1.0,10.1093/ndt/gft319,10.1093/ndt/gft319
1266,1,1.0,10.1093/cid/ciu795,10.1093/cid/ciu795
1265,1,1.0,10.1093/cid/ciu795,10.1093/cid/ciu795
1190,1,1.0,10.1093/cid/cir669,10.1093/cid/cir669
165879,0,1.0,10.1093/cid/cir669,10.1093/cid/cir669
1264,1,1.0,10.1093/ndt/gft319,10.1093/ndt/gft319
1192,1,1.0,10.1093/cid/cir669,10.1093/cid/cir669
1260,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
1256,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
1257,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
162614,0,0.0,10.1007/978-3-642-41698-9,
200248,0,0.0,10.1007/978-3-642-41698-9,
168016,0,0.0,10.1093/cid/ciu795,
207697,0,0.0,10.1093/cid/ciu795,
166271,0,0.0,10.1093/cid/cir669,
207329,0,0.0,10.1093/cid/ciu795,
200281,0,0.0,10.1007/978-3-642-41698-9,
255312,0,0.0,10.1055/b-005-143650,
255360,0,0.0,10.1055/b-005-143650,
208591,0,0.0,10.1093/ndt/gft319,


doi_delta = 0.0


As can be seen above, a value of 1.0 is returned if both strings of a pair are empty. As the attribute filling of $\texttt{doi}$ is sparse, see chapter [Data Analysis](./1_DataAnalysis.ipynb), the $\texttt{doi}\_\texttt{delta}$ indicates strongly a pair of duplicates for most rows. To avoid such misleading identity indication, function $\texttt{.mark}\_\texttt{missing()}$ will be applyed to the attribute.

In [23]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### edition

In its original form in Swissbib's raw data, the edition statement is a string value which may have several words. The modelling on this attribute has been tried with and without stripping letter characters from the string. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). A Jaccard similarity is tried for this attribute.

In [24]:
attribute = 'edition'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [25]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

import numpy as np

edition_delta_uniques = np.sort(df_feature_base['edition_delta'].unique())
edition_delta_uniques_len = len(edition_delta_uniques)
print('edition values range', edition_delta_uniques[:30])

edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]
edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]


The comparison results in a wide number of distinct similarity values for the goldstandard data set. Below, some examples are shown.

In [26]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position],
    edition_delta_uniques[edition_delta_uniques_len-position+2], 10)

position = edition_delta_uniques_len//2

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
200573,0,1.0,,
214405,0,1.0,,
218889,0,1.0,,
22640,0,1.0,,
113695,0,1.0,,
155265,0,1.0,,
74630,0,1.0,,
159447,0,1.0,,
168017,0,1.0,,
28334,0,1.0,,


0.6 <= edition_delta <= 1.0


,duplicates,edition_delta,edition_x,edition_y
145441,0,0.0,3,
225243,0,0.0,,4
57856,0,0.0,19,
123605,0,0.0,2,
84689,0,0.0,5,1994
14613,0,0.0,,1905
254460,0,0.0,19,
32035,0,0.0,,1994
215647,0,0.0,6,
201590,0,0.0,,4


0.0 <= edition_delta <= 0.1428571428571428


,duplicates,edition_delta,edition_x,edition_y
248432,0,0.20,10425,2
184341,0,0.25,1863,3
192294,0,0.25,2,2007
103344,0,0.20,10,2007
190137,0,0.25,6,1869
184569,0,0.25,1863,8
102969,0,0.20,10,1899
253326,0,0.25,7,1907
96959,0,0.20,1791,87
248226,0,0.20,10425,4


0.19999999999999996 <= edition_delta <= 0.25


Again, for $\texttt{edition}\_\texttt{delta} = 1$, many empty values of the $\texttt{edition}$ attribute can be observed. These will be marked with the special negative value in the data with the goal to distinguish them from the matching attribute pairs.

In [27]:
df_feature_base = dpf.mark_missing(df_feature_base, 'edition', factor)

In [28]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
83893,0,1.0,5,5
101350,0,1.0,2,2
71899,0,1.0,8,8
181000,0,1.0,6,6
109608,0,1.0,4,4
1077,1,1.0,3,3
226810,0,1.0,3,3
157823,0,1.0,6,6
209103,0,1.0,6,6
188877,0,1.0,3,3


0.6 <= edition_delta <= 1.0


### exactDate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{exactDate}$ holds a year number stored in the first four digits. Letter 'u' is used as a placeholder for an unknown digit. The attribute may hold some month and day or a second year information in its second four digits, additionally.

The attribute will be kept as a string and will not be transformed to an integer. The feature attribute of the record pair to be compared will be calculated with a modified Hamming algorithm, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The resulting similarity will be stored in a new attribute $\texttt{exactDate}\_\texttt{delta}$ which will be taken for the model calculation.

As can be seen in chapter [Decision Tree Model](./6_DecisionTreeModel.ipynb), this attribute is important for prediction. Different ways of increasing the weight of the unknown status of a digit have been tried. The different ways can be seen in the implementations below. The algorithm decided for the final simulation will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb).

In [29]:
import string

def no_xor (x_side, y_side) :
    number = 0
    for i in range(len(x_side)) :
        if ((x_side[i] in string.ascii_lowercase) | (y_side[i] in string.ascii_lowercase)) & (x_side[i] != y_side[i]) :
            number = number + 1
    return number

print('Example comparison results in a value of', no_xor ('202a0aaa', '1920uuuu'))

Example comparison results in a value of 5


In [30]:
attribute = 'exactDate'

# Replace letter 'u' with letter 'a' for one of the two strings.
#  As an effect, the resulting Hamming similarity for a letter
#  instead of a numerical digit in either string will add with an amount 0.
df_feature_base[attribute+'_x'] = df_feature_base.exactDate_x.str.replace('u', 'a')

# Compute Hamming similarity for century string pair.
columns_metadata_dict['similarity_metrics'][attribute] = tedi.Hamming()

unknown_share = 16

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

if exactDate_mode == 'added_u':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only maximum number of letter digits in both strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        max(x[attribute+'_x'].count('a'), x[attribute+'_y'].count('u'))/unknown_share, axis=1
    )
elif exactDate_mode == 'xor':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only number of position-wise xor-ed letter digits in the two strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        no_xor(x[attribute+'_x'], x[attribute+'_y'])/unknown_share, axis=1
    )

In [31]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']].sample(n=10)

,exactDate_x,exactDate_y,exactDate_delta
209254,2013aaaa,1990uuuu,0.250
99925,2005aaaa,2005uuuu,0.750
98431,2005aaaa,1997uuuu,0.250
199461,2016aaaa,uuuuuuuu,0.500
192023,2016aaaa,1998uuuu,0.250
113456,2010aaaa,19669999,0.250
193620,2016aaaa,1995uuuu,0.250
105481,1984aaaa,1982uuuu,0.625
29241,2000aaaa,2012uuuu,0.500
241589,2016aaaa,1988uuuu,0.250


All resulting values of equal strings are equal to 1.

In [32]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']][
    df_feature_base.exactDate_x == df_feature_base.exactDate_y
].sort_values('exactDate_delta', ascending=False).head()

,exactDate_x,exactDate_y,exactDate_delta
159,20022000,20022000,1.0
843,19849999,19849999,1.0
845,19969999,19969999,1.0
846,19969999,19969999,1.0
847,19969999,19969999,1.0


A discrete set of different similarity values can be found in the attribute deltas. Some sample records are shown below.

In [33]:
exactDate_deltas = np.sort(df_feature_base.exactDate_delta.unique())
exactDate_deltas

array([0.    , 0.125 , 0.25  , 0.3125, 0.375 , 0.4375, 0.5   , 0.5625,
       0.625 , 0.6875, 0.75  , 0.875 , 1.    ])

In [34]:
sample_size = 5

for i in exactDate_deltas :
    dpf.show_samples_distinct(df_feature_base, 'exactDate', i, sample_size)
    print(f'exactDate_delta = {i}')

,duplicates,exactDate_delta,exactDate_x,exactDate_y
99226,0,0.0,19702006,20151475
174833,0,0.0,20159999,19972008
81728,0,0.0,20071990,19942008
113881,0,0.0,20071990,19282011
9914,0,0.0,20022000,19441999


exactDate_delta = 0.0


,duplicates,exactDate_delta,exactDate_x,exactDate_y
78126,0,0.125,17001799,20022010
154052,0,0.125,18501875,19989999
24499,0,0.125,19001999,20012002
175646,0,0.125,20159999,19571963
153891,0,0.125,18501875,20071991


exactDate_delta = 0.125


,duplicates,exactDate_delta,exactDate_x,exactDate_y
217602,0,0.25,19972002,2002uuuu
53786,0,0.25,2002aaaa,1879uuuu
9025,0,0.25,2000aaaa,1862uuuu
43556,0,0.25,2012aaaa,1993uuuu
45683,0,0.25,2007aaaa,1996uuuu


...

exactDate_delta = 0.75


,duplicates,exactDate_delta,exactDate_x,exactDate_y
136125,0,0.875,19949999,19uu9999
89491,0,0.875,19969999,19989999
50920,0,0.875,19849999,19949999
141625,0,0.875,19972002,19972008
136122,0,0.875,19949999,19959999


exactDate_delta = 0.875


,duplicates,exactDate_delta,exactDate_x,exactDate_y
784,1,1.0,20091990,20091990
1286,1,1.0,20159999,20159999
472,1,1.0,19201929,19201929
1287,1,1.0,20159999,20159999
967,1,1.0,18901897,18901897


exactDate_delta = 1.0


### format

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{format}$ has been split up into two new attributes $\texttt{format}\_\texttt{prefix}$ and $\texttt{format}\_\texttt{postfix}$ which will be compared by a different similarity metrics.

- As the quality of $\texttt{format}\_\texttt{prefix}$ is expected to be high, an identity comparison should be sufficient.
- Due to the observed structure of $\texttt{format}\_\texttt{postfix}$, a q-gram based comparison will be chosen.

In [35]:
attribute = 'format'

columns_metadata_dict['similarity_metrics'][attribute+'_prefix'] = tedi.Identity()
columns_metadata_dict['similarity_metrics'][attribute+'_postfix'] = tedi.Jaccard(qval=2)

pfix_values = ['_prefix', '_postfix']

for pf in pfix_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pf,
        columns_metadata_dict['similarity_metrics'][attribute+pf],
        columns_metadata_dict)

In [36]:
for i in df_feature_base.format_prefix_delta[
    df_feature_base.format_prefix_x != df_feature_base.format_prefix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_prefix', i)
    print(f'format_prefix_delta = {i}')

,duplicates,format_prefix_delta,format_prefix_x,format_prefix_y
49824,0,0.0,bk,vm
106646,0,0.0,bk,vm
160769,0,0.0,vm,bk
66251,0,0.0,bk,mu
174350,0,0.0,bk,mu


format_prefix_delta = 0.0


In [37]:
for i in df_feature_base.format_postfix_delta[
    df_feature_base.format_postfix_x != df_feature_base.format_postfix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_postfix', i)
    print(f'format_postfix_delta = {i}')

,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
183867,0,0.428571,020800,020000
168290,0,0.428571,030600,010300
193358,0,0.428571,020300,020000
75811,0,0.428571,010200,020000
163253,0,0.428571,020000,010000


format_postfix_delta = 0.4285714285714286


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
23863,0,0.111111,010300,020053
153162,0,0.111111,010200,040000
10983,0,0.111111,040100,030300
205993,0,0.111111,020053,010000
135919,0,0.111111,020000,010300


format_postfix_delta = 0.11111111111111116


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
145433,0,0.25,020000,020353
206130,0,0.25,020000,020353
197093,0,0.25,010347,010100
196625,0,0.25,010347,040100
156318,0,0.25,020000,020353


...

format_postfix_delta = 0.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
199204,0,1.0,020000,020000
61565,0,1.0,020000,020000
185,1,1.0,030600,030600
101783,0,1.0,010300,010300
112977,0,1.0,020000,020000


format_postfix_delta = 1.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
245199,0,0.666667,020400,020047
88975,0,0.666667,030500,030053


format_postfix_delta = 0.6666666666666666


### isbn

Swissbib uses each string element of the $\texttt{isbn}$ list separately for comparing with each string element of its comparison $\texttt{isbn}$ list. If two bibliographic units hold at least one element in common, this is interpreted as a strong indicator for duplicates [[WiCo2001](./A_References.ipynb#wico2001)].

This hard logic is used in a modified way in the context of this capstone project. A special comparison function $\texttt{.build}\_\texttt{delta}\_\texttt{isbn()}$ has been implemented that compares each list element of the left-hand side with each list element of the right-hand side of a pair. According to Swissbib's implementation, the Identity metric is used for string comparison, calculating a similarity value of 1.0 or 0.0 for each list element pair. For normalisation reasons, the sum of similarity values is divided by the number of elements of the smaller list. If both lists are empty a value of 1.0 is returned. If only one list is empty a value of 0.0 is returned.

In [38]:
attribute = 'isbn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base[attribute+'_delta'].unique()

array([1. , 0. , 0.5])

Some sample cases are shown below for each category of $\texttt{isbn_delta}$.

In [39]:
for isbn_delta_value in df_feature_base['isbn_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['isbn_delta']==isbn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'isbn', isbn_delta_value, number_of_max_samples)
    print(f'isbn_delta = {isbn_delta_value}')

,duplicates,isbn_delta,isbn_x,isbn_y
156083,0,1.0,[],[]
89767,0,1.0,[],[]
100673,0,1.0,[],[]
43723,0,1.0,[],[]
51184,0,1.0,[],[]
243132,0,1.0,[],[]
230431,0,1.0,[],[]
213113,0,1.0,[],[]
169345,0,1.0,[],[]
144163,0,1.0,[],[]


isbn_delta = 1.0


,duplicates,isbn_delta,isbn_x,isbn_y
106386,0,0.0,[],"[978-3-598-31511-4 (print), 978-3-11-096917-7]"
164513,0,0.0,[],"[978-3-598-31497-1 (print), 978-3-11-097083-8]"
33061,0,0.0,[3-495-47879-5],[3-7618-0609-4]
66441,0,0.0,[],[0258-0683]
237113,0,0.0,"[2-07-042871-0 (Br), 978-2-07-042871-7 (Br)]",[0-19-254704-6]
235319,0,0.0,[],"[3-906721-36-1 (Livre de l'élève, inkl. Villag..."
92353,0,0.0,[],"[3-7957-6954-X, 978-3-7957-6954-3]"
204952,0,0.0,[],"[3-492-18008-6, 3-7957-8008-X]"
241330,0,0.0,[978-0-7294-1155-4],[3-292-00371-7]
51885,0,0.0,[3-932992-42-3],[3-15-002620-2]


isbn_delta = 0.0


,duplicates,isbn_delta,isbn_x,isbn_y
1195,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1205,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1199,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1201,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1202,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1210,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"


isbn_delta = 0.5


For attribute $\texttt{isbn}$, the special marking of missing values is omitted.

### ismn

This attribute will be processed with the identity similarity metric. The reasoning for this decision is the same as for similar attributes above. 

In [40]:
attribute = 'ismn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [41]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

ismn values range [0. 1.]


In [42]:
for ismn_delta_value in df_feature_base[attribute+'_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base[attribute+'_delta']==ismn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'ismn', ismn_delta_value, number_of_max_samples)
    print(f'ismn_delta = {ismn_delta_value}')

,duplicates,ismn_delta,ismn_x,ismn_y
19355,0,1.0,,
32134,0,1.0,,
211277,0,1.0,,
148220,0,1.0,,
211505,0,1.0,,
195079,0,1.0,,
139181,0,1.0,,
150850,0,1.0,,
208084,0,1.0,,
96104,0,1.0,,


ismn_delta = 1.0


,duplicates,ismn_delta,ismn_x,ismn_y
99251,0,0.0,m006546749,
137185,0,0.0,m006204687,
250599,0,0.0,m006204687,
250738,0,0.0,m006204687,
152590,0,0.0,,9790006201334
249686,0,0.0,9790006450510,
21384,0,0.0,,m006450510
103288,0,0.0,m006450510,
102317,0,0.0,,m700241001
115237,0,0.0,"m006546756 (kritischer bericht, leinen)",


ismn_delta = 0.0


As can be seen in the previous chapters, attribute $\texttt{ismn}$ is filled sparsely. A lot of missing values calculate to a value of 1.0 in the chosen similarity metrics. To mark these cases specifically, they will be transformed to a negative value.

In [43]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### musicid

Chapter [Data Analysis](./1_DataAnalysis.ipynb) shows that attribute $\texttt{musicid}$ is an identifyer for a music record. A Jaccard metric has been tested on this attribute, resulting in a distribution of many high similarity values on uniques. Comparing this result with the LCS metric, the latter has been decided.

In [44]:
attribute = 'musicid'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [45]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

musicid values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.375      0.4        0.42857143 0.5
 0.6        0.66666667 0.75       1.        ]


In [46]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//2

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)

,duplicates,musicid_delta,musicid_x,musicid_y
37082,0,1.0,4553,4553
1021,1,1.0,0630,0630
1049,1,1.0,10003,10003
22,1,1.0,073,073
990,1,1.0,4553,4553
75621,0,1.0,10425,10425
1044,1,1.0,4553,4553
460,1,1.0,422,422
103087,0,1.0,4553,4553
301,1,1.0,502023,502023


0.75 <= musicid_delta <= 1.0


,duplicates,musicid_delta,musicid_x,musicid_y
43576,0,0.0,502430,
181649,0,0.0,,
245760,0,0.0,,
187487,0,0.0,,
33177,0,0.0,,
20761,0,0.0,,
161819,0,0.0,,
182217,0,0.0,,
92825,0,0.0,,
5832,0,0.0,,


0.0 <= musicid_delta <= 0.1428571428571429


,duplicates,musicid_delta,musicid_x,musicid_y
73988,0,0.400,10425,422
73926,0,0.400,10425,542
75767,0,0.400,10425,422
64504,0,0.400,422,10425
229885,0,0.400,10425,542
198364,0,0.375,502430,66467243
43591,0,0.375,502430,66467243
116411,0,0.400,10425,242
134537,0,0.400,10425,8421
68058,0,0.400,10425,422


0.375 <= musicid_delta <= 0.4


In [47]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1], attribute,
    uniques[attribute][0],
    uniques[attribute][uniques_len[attribute]-1], 20)

,duplicates,musicid_delta,musicid_x,musicid_y
1157,1,0.2,063,50999
944,1,0.0,422,
1248,1,0.0,,
1114,1,0.0,,
123,1,0.0,,
479,1,0.0,10425,
97,1,0.0,,
580,1,1.0,99036,99036
1432,1,0.0,,
887,1,0.0,,


0.0 <= musicid_delta <= 1.0


The attribute is filled with a degree of below $10\%$. The chosen metric for it results in a similarity value of 1.0 for empty value pairs. This effect can be adjusted with function $\texttt{.mark}\_\texttt{missing}()$ as above. 

In [48]:
df_feature_base = dpf.mark_missing(df_feature_base, 'musicid', factor)

### part

Analogous to attribute $\texttt{edition}$ described above, the string value of this attribute can be stripped to pure number digits. Both ways, with and without letter stripping have been tried for modelling. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). Three different metrics have been tried for attribute $\texttt{part}$. Finally, metric Jaro will be used.

In [49]:
attribute = 'part'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.Hamming()
#tedi.LCSStr()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [50]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

part values range [0.         0.23651613 0.23917051 ... 0.975      0.98666667 1.        ]


In [51]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//7

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

,duplicates,part_delta,part_x,part_y
211740,0,1.0,,
202423,0,1.0,,
160675,0,1.0,,
69795,0,1.0,,
12462,0,1.0,,
194262,0,1.0,,
45495,0,1.0,,
152144,0,1.0,,
194638,0,1.0,,
2714,0,1.0,,


0.9866666666666667 <= part_delta <= 1.0


,duplicates,part_delta,part_x,part_y
244968,0,0.0,,272(1993)
60858,0,0.0,,"7, 7"
61281,0,0.0,,20 c
49976,0,0.0,,7
103892,0,0.0,,"bl. 23, 23,1988, 23"
170362,0,0.0,4,
250342,0,0.0,,291(2012)
33371,0,0.0,57,
246426,0,0.0,20c,
46273,0,0.0,,"bd. 27, 27"


0.0 <= part_delta <= 0.23917050691244246


,duplicates,part_delta,part_x,part_y
142985,0,0.618519,"2620, 2620",285(2006)
125788,0,0.618519,"2620, 2620",285(2006)
142982,0,0.618519,"2620, 2620",282(2003)
125927,0,0.618519,"2620, 2620",1260/1261
142981,0,0.618519,"2620, 2620",281(2002)
143124,0,0.618519,"2620, 2620",1260/1261
214603,0,0.618687,"no 912, 912","nr. 2620, 2620"
214573,0,0.618687,"no 912, 912","nr. 2620, 2620"
214633,0,0.618687,"no 912, 912","nr. 2620, 2620"
237274,0,0.618519,"3870, 3870",287(2008)


0.6185185185185186 <= part_delta <= 0.6186868686868686


,duplicates,part_delta,part_x,part_y
238029,0,0.619048,"bd. 57, 57","band 4, band 4"
98850,0,0.619048,band 19,[bd.] 7
90459,0,0.619048,band 57,[bd. 1]
191951,0,0.619048,"band 57, ed. 2",[bd.] 7
196524,0,0.619048,"bd. 57, 57","band 8, band 8"
257922,0,0.619048,"bd. 8008, 8008","nr. 2620, 2620"
128923,0,0.619048,"bd. 57, 57","band 9, band 9"
164080,0,0.619048,band 19,[bd.] 7
201261,0,0.619048,"n. 1, 1","band 3, band 3"
128917,0,0.619048,"bd. 57, 57","band 4, band 4"


0.6187301587301587 <= part_delta <= 0.6190476190476191


In this attribute, too, moving pairs of empty values to negative values will result in a clearer distinction between pairs of uniques and duplicates, as will be seen in the graphical comparison of capter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb).

In [52]:
df_feature_base = dpf.mark_missing(df_feature_base, 'part', factor)

### person

As a result of chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{person}$ has been split into three specific attributes. Attribute $\texttt{person}\_{100}$ and $\texttt{person}\_{700}$ hold strongly standardised string values. For comparing pure strings, a Levenshtein metric is recommended [[Chri2012](./A_References.ipynb#chri2012)]. Unfortunately, this metric shows a very long calculation time on the data of the capstone project. Comparing the similarity values of the Levenshtein metric with the similarity values of other metrics in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), similarity metric StrCmp95 has been decided to use.

In [53]:
attribute = 'person'

columns_metadata_dict['similarity_metrics'][attribute+'_100'] = tedi.StrCmp95()
columns_metadata_dict['similarity_metrics'][attribute+'_700'] = tedi.StrCmp95()
#tedi.Levenshtein()

pe_values = ['_100', '_700']

for pe in pe_values :
    print('Calculating person'+pe)
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pe,
        columns_metadata_dict['similarity_metrics'][attribute+pe],
        columns_metadata_dict)

Calculating person_100


Calculating person_700


In [54]:
pe = '_100'

uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_100 values range [0.         0.31944444 0.32777778 0.33095238 0.34555556 0.34768519
 0.3562963  0.35714286 0.36414141 0.36527778 0.36723647 0.37777778
 0.38511815 0.38603989 0.38779956 0.38989899 0.39246032 0.39490741
 0.40142857 0.40347222 0.40666667 0.41281046 0.4130719  0.41596491
 0.42324786 0.42388889 0.4240404  0.42486772 0.42614379 0.42620321
 0.42676768 0.42819444 0.42869281 0.42936508 0.42982456 0.42991453
 0.43208333 0.43326118 0.43415893 0.43508772 0.43555556 0.43590643
 0.43660714 0.43794872 0.43804714 0.439819   0.44040404 0.44079254
 0.44126984 0.44222222 0.44282407 0.44292929 0.44444444 0.44457516
 0.44507937 0.44571078 0.44649123 0.44656085 0.4465812  0.44662698
 0.44722222 0.44782135 0.44786325 0.44791667 0.4481352  0.44861111
 0.44972222 0.45126263 0.45131313 0.45222222 0.45291375 0.45314327
 0.45339367 0.45416667 0.45424242 0.45555556 0.45564103 0.45578704
 0.45614973 0.45784314 0.45873016 0.45952381 0.46013072 0.46055556
 0.46119281 0.46127946 0.46336996 0.46

In [55]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_100_delta,person_100_x,person_100_y
244423,0,1.0,,
132713,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
127673,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
139961,0,1.0,,
86749,0,1.0,,
119864,0,1.0,,
232470,0,1.0,,
22415,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
90260,0,1.0,,
174356,0,1.0,,


0.8666666666666666 <= person_100_delta <= 1.0


,duplicates,person_100_delta,person_100_x,person_100_y
233650,0,0.0,,mozartwolfgang amadeus
237213,0,0.0,voltaire,
98506,0,0.0,,bührerwalter
30591,0,0.0,,mozartwolfgang amadeus
195176,0,0.0,broderursula,
101823,0,0.0,mozartwolfgang amadeus,
215191,0,0.0,voltaire,
194611,0,0.0,broderursula,
219034,0,0.0,,mozartwolfgang amadeus
257781,0,0.0,mozartwolfgang amadeus,


0.0 <= person_100_delta <= 0.3277777777777777


For comparing person names, like in attribute $\texttt{person}\_{245c}$, a Jaro metric will be tested [[Chri2012](./A_References.ipynb#chri2012)].

In [56]:
pe = '_245c'

columns_metadata_dict['similarity_metrics'][attribute+pe] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute+pe,
    columns_metadata_dict['similarity_metrics'][attribute+pe],
    columns_metadata_dict)

In [57]:
uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_245c values range [0.         0.24632035 0.25462963 ... 0.99583333 0.99603175 1.        ]


In [58]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_245c_delta,person_245c_x,person_245c_y
89936,0,1.0,,
122668,0,1.0,sigrid kessler... [et al.] ; [éd.:] interkanto...,sigrid kessler... [et al.] ; [éd.:] interkanto...
201366,0,1.0,,
374,1,1.0,mozart,mozart
232831,0,1.0,,
243823,0,1.0,,
190717,0,1.0,ein film von luc jacquet,ein film von luc jacquet
54915,0,1.0,,
1032,1,1.0,"hans-peter schuster, hans-joachim trappe","hans-peter schuster, hans-joachim trappe"
5838,0,1.0,,


0.996031746031746 <= person_245c_delta <= 1.0


,duplicates,person_245c_delta,person_245c_x,person_245c_y
69382,0,0.0,,sigrid kessler [u.a.]
3506,0,0.0,,sigrid kessler... [et al.] ; [hrsg.:] interkan...
201792,0,0.0,,[hrsg.] interkantonale lehrmittelzentrale rapp...
61898,0,0.0,von mozart ; dichtung von emanuel schikaneder ...,
96092,0,0.0,,"[birgit schenk, gerhard schwabe]"
258416,0,0.0,ein film von luc jacquet,
232401,0,0.0,,sigrid kessler ; [éds.] interkantonale lehrmit...
211457,0,0.0,,jane austen
233253,0,0.0,,liane moriarty ; trad. de l'anglais (australie...
89517,0,0.0,,von w.a. mozart ; für pianoforte zu vier hände...


0.0 <= person_245c_delta <= 0.25462962962962954


The similarities of all three $\texttt{person}$ attributes are affected by empty values. These will be handled the same way as the attributes above.

In [59]:
pe_values = ['_100', '_245c', '_700']

for pe in pe_values :
    df_feature_base = dpf.mark_missing(df_feature_base, 'person'+pe, factor)

### pubinit

This attribute holds publisher strings that have a similar representation as attribute $\texttt{person}$. A Jaro metric will be used.

In [60]:
attribute = 'pubinit'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [61]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

pubinit values range [0.         0.25132275 0.25303644 ... 0.98404165 0.98484848 1.        ]


In [62]:
position = uniques_len[attribute]//3

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-1],
    uniques[attribute][uniques_len[attribute]-position], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-5],
    uniques[attribute][uniques_len[attribute]-1], 10)

,duplicates,pubinit_delta,pubinit_x,pubinit_y
45003,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
97706,0,0.554626,"tdk recording media europe, opernhaus",p. reclam jun.
97782,0,0.554626,"tdk recording media europe, opernhaus",p. reclam jun.
199776,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
199772,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
2900,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
149367,0,0.554701,erato,"interkantonale lehrmittelzentrale ; [bern], be..."
2896,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
90664,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
90660,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."


0.5546260546260546 <= pubinit_delta <= 0.5547008547008546


,duplicates,pubinit_delta,pubinit_x,pubinit_y
59385,0,1.0,,
258379,0,1.0,,
245346,0,1.0,,
17257,0,1.0,,
84299,0,1.0,,
169213,0,1.0,,
179172,0,1.0,,
239666,0,1.0,,
20212,0,1.0,,
76322,0,1.0,,


0.9777777777777779 <= pubinit_delta <= 1.0


The similarities of $\texttt{pubinit}$ is affected by empty values. These will be transformed to negative values.

In [63]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### scale

Comparing the similarity metrics of some sample value pairs of attribute $\texttt{scale}$ in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), a Jaccard metrics has been identifyed to express the best matching behaviour for purely numerical values stored in the attribute.

In [64]:
attribute = 'scale'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [65]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

scale values range [0.         0.04587156 0.05504587 0.57142857 1.        ]


In [66]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,scale_delta,scale_x,scale_y
127600,0,0.0,,100000
2161,0,0.0,,100000
62984,0,0.0,,100000
80169,0,0.0,,100000
55094,0,0.0,50000,
45765,0,0.0,,100000
103562,0,0.0,,100000
233179,0,0.0,,100000
184305,0,0.0,100000,
207925,0,0.0,,100000


0.0 <= scale_delta <= 0.04587155963302747


,duplicates,scale_delta,scale_x,scale_y
96192,0,0.571429,50000,100000
96179,0,0.571429,50000,100000
182760,0,0.571429,50000,100000
182430,0,0.571429,50000,100000
69503,0,0.571429,50000,100000
227504,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
182753,0,0.571429,50000,100000
96176,0,0.571429,50000,100000
95855,0,0.571429,50000,100000
182754,0,0.571429,50000,100000


0.05504587155963303 <= scale_delta <= 0.5714285714285714


,duplicates,scale_delta,scale_x,scale_y
227816,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
227818,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
227823,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
227863,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
227841,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
227832,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
227830,0,0.045872,Scala 1:50.000 ; proiezione cilindrica ad asse...,50000
227501,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
227815,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
227500,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000


0.04587155963302747 <= scale_delta <= 0.05504587155963303


Attribute $\texttt{scale}$ is filled for maps, only. Due to its sparse filling, the similarities of the attribute are affected strongly by empty values. These empty values will be marked with a special negative value.

In [67]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### ttlfull

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{ttlfull}$ has been split up into two new attributes $\texttt{ttlfull_245}$ and $\texttt{ttlfull_246}$ which will be compared by the same similarity metrics. A visual analysis of the values stored in the attribute, reveals a string of words, comparable to the strings in attribute $\texttt{person_245c}$, above. The same similarity metric will be used for both title attributes, therefore.

In [68]:
attribute = 'ttlfull'

columns_metadata_dict['similarity_metrics'][attribute+'_245'] = tedi.Jaro()
columns_metadata_dict['similarity_metrics'][attribute+'_246'] = tedi.Jaro()

tf_values = ['_245', '_246']

for tf in tf_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+tf,
        columns_metadata_dict['similarity_metrics'][attribute+tf],
        columns_metadata_dict)

In [69]:
for tf in tf_values :
    uniques[attribute+tf], uniques_len[attribute+tf] = dpf.determine_similarity_values(
        df_feature_base, attribute+tf)

ttlfull_245 values range [0.         0.24175824 0.24358974 ... 0.99555556 0.99747475 1.        ]
ttlfull_246 values range [0.         0.27407407 0.35555556 0.36111111 0.36366612 0.36666667
 0.39444444 0.39814815 0.40238095 0.40555556 0.41140642 0.41190476
 0.41296296 0.41313131 0.4168847  0.41923077 0.41931217 0.42380952
 0.42424242 0.42484886 0.42767857 0.43003072 0.43333333 0.43758865
 0.43913043 0.43998145 0.44198966 0.44242424 0.44507736 0.44539295
 0.44722222 0.4479837  0.44848485 0.44901497 0.45       0.45011338
 0.45277778 0.45436508 0.45610717 0.45714286 0.45764895 0.45833333
 0.46094276 0.46205962 0.46527102 0.46729958 0.46851852 0.46885522
 0.46944444 0.46945938 0.47130647 0.47142857 0.47162447 0.47222222
 0.472318   0.47267613 0.47285513 0.47308219 0.47356964 0.47372609
 0.47388167 0.47417027 0.47428613 0.47431581 0.47446677 0.47462215
 0.47491364 0.47508251 0.47515152 0.47518634 0.47597067 0.47606838
 0.47613256 0.47629884 0.47642654 0.47794389 0.47829216 0.47836071
 0.4788

In [70]:
tf = '_245'
position = uniques_len[attribute+tf]

dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-position],
    uniques[attribute+tf][uniques_len[attribute+tf]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-3],
    uniques[attribute+tf][uniques_len[attribute+tf]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-4],
    uniques[attribute+tf][uniques_len[attribute+tf]-3], 10)

,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
37836,0,0.0,bildungsforschung und bildungspraxis,emma
4197,0,0.0,emma,blick in die welt
108114,0,0.0,emma,trionfi
107985,0,0.0,emma,blick in die welt
137642,0,0.0,emma,blick in die welt
219182,0,0.0,bildungsforschung und bildungspraxis,emma
33981,0,0.0,arts,emma
96440,0,0.0,emma,how do i find the christ?
33807,0,0.0,arts,emma
219219,0,0.0,bildungsforschung und bildungspraxis,emma


0.0 <= ttlfull_245_delta <= 0.2417582417582418


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
237,1,0.997475,sozialleistungsbetrug: sozialversicherungsbetr...,sozialleistungsbetrug: sozialversicherungsbetr...
249,1,0.997475,sozialleistungsbetrug: sozialversicherungsbetr...,sozialleistungsbetrug: sozialversicherungsbetr...
144730,0,0.995556,"bonne chance !, cours de langue française deux...","bonne chance !, cours de langue française, deu..."


0.9955555555555556 <= ttlfull_245_delta <= 0.9974747474747474


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
32,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
29,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
90,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
194138,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
1349,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
238020,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
90956,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
88,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
64,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
1351,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."


0.9953703703703703 <= ttlfull_245_delta <= 0.9955555555555556


Attribute $\texttt{ttlfull}\_\texttt{245}$ is filled for all data rows of Swissbib's raw data as can be seen in chapter [Data Analysis](./1_DataAnalysis.ipynb). For attribute $\texttt{ttlfull}\_\texttt{245}$, the filling is below $10\%$. The data pairs with missing values will be marked with a negative value as has been done for similar cases above.

In [71]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute+'_246', factor)

### volumes

This attribute is described in chapter [Data Analysis](./1_DataAnalysis.ipynb) to hold a kind of contents that resembles the contents of attribute $\texttt{part}$. The same similarity metrics will be used for attribute $\texttt{volumes}$ as for attribute $\texttt{part}$, therefore.

In [72]:
attribute = 'volumes'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.LCSSeq()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [73]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

volumes values range [0.         0.25632184 0.25714286 ... 0.98484848 0.98765432 1.        ]


In [74]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,volumes_delta,volumes_x,volumes_y
177396,0,0.0,,453 p.
219254,0,0.0,v.,367 s.
149937,0,0.0,3 cd,
227271,0,0.0,"iv, 191 p.",
24957,0,0.0,64 s.,
181820,0,0.0,bd.,600 s.
169362,0,0.0,2 compact discs,
122067,0,0.0,v.,72 p.
147707,0,0.0,dvd-rom,
50759,0,0.0,v.,37 s.


0.0 <= volumes_delta <= 0.2563218390804598


,duplicates,volumes_delta,volumes_x,volumes_y
74548,0,0.984848,1 dvd-video (169 min.),1 dvd-video (16 min.)
283,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
295,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
307,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
286,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
271,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
274,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
292,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
304,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
101826,0,0.984848,1 dvd-video (169 min.),1 dvd-video (16 min.)


0.9848484848484849 <= volumes_delta <= 0.9876543209876543


,duplicates,volumes_delta,volumes_x,volumes_y
101826,0,0.984848,1 dvd-video (169 min.),1 dvd-video (16 min.)
74548,0,0.984848,1 dvd-video (169 min.),1 dvd-video (16 min.)
149395,0,0.978495,2 disques compacts en 1 coffret,3 disques compacts en 1 coffret


0.978494623655914 <= volumes_delta <= 0.9848484848484849


Attribute $\texttt{volumes}$ holds rows with missing data. The data pairs with missing values will be marked with a special negative value.

In [75]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

## DataFrame with Attributes and Similarity Features

The metric for each attribute of the feature DataFrame has been decided and the similarity features have been calculated. In this last step, the columns of the DataFrame are reordered in order to place the $\_\texttt{delta}$ columns close to their input origins $\_\texttt{x}$ and $\_\texttt{y}$ and some sample records are shown.

In [76]:
# Take _x, _y, and _delta columns together
fb_col_list = df_feature_base.columns.tolist()
fb_col_list.sort()
# Move target column to first place
fb_col_list.insert(0, fb_col_list.pop(fb_col_list.index('duplicates')))
# Reorder DataFrame columns
df_attribute_with_sim_feature = pd.DataFrame(df_feature_base, columns=fb_col_list)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_attribute_with_sim_feature.columns)

class_label = ['uniques', 'duplicate']

for i in class_label:
    display(df_attribute_with_sim_feature[df_attribute_with_sim_feature.duplicates==class_label.index(i)].sample(n=10))
    print(i)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
179086,0,-0.10,,,-0.10,,,-0.10,,,-0.10,,,-0.10,,,0.250,2007aaaa,1996uuuu,1.000000,020000,020000,1.0,bk,bk,0.0,"[978-3-7815-1531-4, 3-7815-1531-1]",[],-0.10,,,-0.10,,,-0.10,,,-0.05,,kesslersigrid,0.529509,"michaela gläser-zikuda, tina hascher (hrsg.)",sigrid kessler [u.a.],-0.050000,"gläser-zikudamichaela, haschertina",,-0.050000,,staatlicher lehrmittelverlag,-0.10,,,0.664208,"lernprozesse dokumentieren, reflektieren und b...","bonne chance!, cours de langue française, nouv...",-0.10,,,0.777778,304 s.,239 s.
232304,0,-0.10,,,-0.10,,,-0.05,schweizerische normen-vereinigung,,-0.10,,,-0.05,,4,0.625,2014aaaa,2010uuuu,0.428571,020053,020000,1.0,bk,bk,0.0,[],[978-3-13-100294-5],-0.10,,,-0.10,,,-0.10,,,-0.10,,,-0.050000,,hrsg. von hans-konrad biesalski ... [et al.],-0.050000,,biesalskihans-konrad,-0.100000,,,-0.10,,,0.559589,informatique de santé - communication entre di...,"ernährungsmedizin, nach dem neuen curriculum e...",-0.05,medizinische informatik - kommunikation von ge...,,0.545476,1 ressource en ligne,1132 s.
137522,0,-0.10,,,-0.10,,,-0.10,,,-0.10,,,-0.10,,,0.250,2005aaaa,1997uuuu,0.428571,020000,030000,0.0,bk,vm,0.0,[0-521-82437-0],[],-0.10,,,-0.10,,,-0.10,,,-0.05,austenjane,,0.591871,jane austen ; ed. by richard cronin and doroth...,[éds.] interkantonale lehrmittelzentrale luzer...,0.651282,croninrichard,kesslersigrid,0.546032,cambridge univ. press,berner lehrmittel- und medienverlag,-0.10,,,0.515504,emma,"bonne chance!, cours de langue française, 2",-0.10,,,0.492308,"lxxvii, 600 p.",médias divers
203612,0,-0.10,,,-0.10,,,-0.05,schweizerische normen-vereinigung,,-0.10,,,-0.10,,,0.500,2011aaaa,uuuuuuuu,0.111111,020053,010000,0.0,bk,mu,1.0,[],[],-0.10,,,-0.05,,140,-0.10,,,-0.05,,mozartwolfgang amadeus,-0.050000,,w. a. mozart,-0.050000,,mozartwolfgang amadeus,0.432336,schweizerische normen-vereinigung (snv),breitkopf,-0.10,,,0.526282,health informatics - personal health device co...,"die zauberflöte, [extraits pour choeur]",-0.05,medizinische informatik - kommunikation von ge...,,-0.050000,,1 matériel de choeur (65 parties)
244482,0,-0.10,,,-0.10,,,-0.05,schweizerische normen-vereinigung,,-0.10,,,-0.10,,,0.625,2016aaaa,2017uuuu,0.111111,020053,030100,0.0,bk,mu,1.0,[],[],-0.10,,,-0.10,,,-0.10,,,-0.05,,bumandominique de,-0.050000,,journaliste simon matthey-doret,-0.050000,,matthey-doretsimon,-0.050000,,la première (radio télévision suisse),-0.10,,,0.556372,informatique de la santé - communication entre...,"dominique de buman, président de l'association...",-0.05,medizinische informatik - kommunikation von ge...,,0.589557,1 ressource en ligne,1 disque compact (21 min.)
167391,0,-0.10,,,-0.10,,,-0.05,,schweizerische normen-vereinigung,-0.05,10.1007/978-3-642-41698-9,,-0.10,,,0.625,2014aaaa,2013uuuu,1.000000,020053,020053,1.0,bk,bk,0.0,[978-3-642-41698-9],[],-0.10,,,-0.10,,,-0.10,,,-0.10,,,-0.050000,birgit spinath (hrsg.),,-0.050000,spinathbirgit,,-0.100000,,,-0.10,,,0.541796,"empirische bildungsforschung, aktuelle themen ...",health informatics - personal health device co...,-0.05,,medizinische informatik - kommunikation von ge...,1.000000,1 online-ressource,1 online-ressource
141851,0,-0.05,,e0074147,-0.05,,n0460833,0.05,"interkantonale lehrmittelz

uniques


,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
504,1,-0.1,,,-0.1,,,-0.10,,,-0.1,,,-0.05,10425,,0.750,1932aaaa,1932uuuu,1.0,010200,010200,1.0,mu,mu,1.0,[],[],-0.1,,,-0.05,,10425,-0.1,,,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus,0.711696,w.a: mozart / [ hrsg. von kurt soldan],w.a. mozart ; ... hrsg. von kurt soldan ; klav...,1.000000,"soldankurt, mozartwolfgang amadeus","soldankurt, mozartwolfgang amadeus",1.000000,c.f. peters,c.f. peters,-0.1,,,1.000000,"die zauberflöte, oper in 2 aufzügen","die zauberflöte, oper in 2 aufzügen",-0.1,,,0.927778,1 réduction (188 p.),"1 réduction (vi, 188 p.)"
898,1,-0.1,,,-0.1,,,-0.10,,,-0.1,,,-0.10,,,0.625,18aaaaaa,18uuuuuu,1.0,010200,010200,1.0,mu,mu,1.0,[],[],-0.1,,,-0.10,,,1.0,lfg. 2,lfg. 2,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus,-0.100000,,,-0.100000,,,-0.100000,,,-0.1,,,1.000000,"die zauberflöte, grosse oper in zwei aufzügen ...","die zauberflöte, grosse oper in zwei aufzügen ...",-0.1,,,1.000000,klavierauszug,klavierauszug
153,1,-0.1,,,-0.1,,,-0.05,"opernhaus (zürich)orchester, opernhaus (zürich...",,-0.1,,,-0.10,,,0.750,1988aaaa,1988uuuu,1.0,040100,040100,1.0,mu,mu,1.0,[],[],-0.1,,,-0.05,242,,-0.1,,,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus,0.720721,mozart ; [livret emanuel schikaneder],mozart,0.897436,"harnoncourtnikolaus, schikanederemanuel",harnoncourtnikolaus,1.000000,teldec,teldec,-0.1,,,0.884058,"die zauberflöte, kv 620",die zauberflöte,-0.1,,,0.639660,"2 disques compacts ddd (73 min. 17 sec., 70 mi...",2 compact discs in box
934,1,-0.1,,,-0.1,,,-0.10,,,-0.1,,,-0.10,,,0.500,aaaaaaaa,uuuuuuuu,1.0,010200,010200,1.0,mu,mu,1.0,[],[],-0.1,,,1.00,245,245,-0.1,,,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus,1.000000,von emanuel schikaneder ; [musik von] wolfgang...,von emanuel schikaneder ; [musik von] wolfgang...,1.000000,"kienzlwilhelm, schikanederemanuel","kienzlwilhelm, schikanederemanuel",-0.100000,,,-0.1,,,1.000000,"die zauberflöte (il flauto magico), oper in zw...","die zauberflöte (il flauto magico), oper in zw...",1.0,"die zauberflöte, ausgabe für gesang und klavier","die zauberflöte, ausgabe für gesang und klavier",1.000000,1 klavierauszug,1 klavierauszug
1150,1,-0.1,,,-0.1,,,-0.10,,,-0.1,,,-0.10,,,0.750,2006aaaa,2006uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[2-226-17006-5],[2-226-17006-5],-0.1,,,-0.10,,,-0.1,,,1.0,rosoffmeg,rosoffmeg,1.000000,meg rosoff ; trad. de l'anglais par hélène collon,meg rosoff ; trad. de l'anglais par hélène collon,1.000000,collonhélène,collonhélène,1.000000,albin michel,albin michel,-0.1,,,1.000000,"maintenant, c'est ma vie","maintenant, c'est ma vie",-0.1,,,1.000000,239 p.,239 p.
172,1,-0.1,,,-0.1,,,0.49,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforsch...,-0.1,,,-0.10,,,1.000,19791999,19791999,1.0,030600,030600,1.0,cr,cr,1.0,[0252-9955],[0252-9955],-0.1,,,-0.10,,,-0.1,,,-0.1,,,0.755807,[hrsg.:] schweizerische gesellschaft für bildu...,hrsg. von der schweizerischen gesellschaft für...,-0.100000,,,0.843137,"universitätsverlag, klett + balmer",universitätsverlag,-0.1,,,0.798535,"bildungsforschung und bildungspraxis, schweize...",bildungsforschung und bildungspraxis,-0.1,,,-0.050000,v.,
819,1,-0.1,,,-0.1,,,1.00,"berliner philharmoniker, deutsche ope

duplicate


## Summary

This chapter covers the central area of feature construction. The features of the feature matrix have been generated for each attribute of Swissbib's raw data, deciding on its similarity metric. With these metric values, the feature base DataFrame has been extended and a new DataFrame with the attribute values of the pairs together with their calculated similarity value have been generated. The similarity values will be the final features for training and performance testing of the models, compare [[JudACaps](./A_References.ipynb#judacaps)].

In [77]:
columns_metadata_dict['similarity_metrics']

{'coordinate_E': LCSStr({'qval': 1, 'external': True}),
 'coordinate_N': LCSStr({'qval': 1, 'external': True}),
 'corporate_full': LCSStr({'qval': 1, 'external': True}),
 'doi': Identity({'qval': 1, 'external': True}),
 'edition': Jaccard({'qval': 1, 'as_set': False, 'external': True}),
 'exactDate': Hamming({'qval': 1, 'test_func': <function Base._ident at 0x1215c1598>, 'truncate': False, 'external': True}),
 'format_prefix': Identity({'qval': 1, 'external': True}),
 'format_postfix': Jaccard({'qval': 2, 'as_set': False, 'external': True}),
 'isbn': Identity({'qval': 1, 'external': True}),
 'ismn': Identity({'qval': 1, 'external': True}),
 'musicid': LCSStr({'qval': 1, 'external': True}),
 'part': StrCmp95({'long_strings': False, 'external': True}),
 'person_100': StrCmp95({'long_strings': False, 'external': True}),
 'person_700': StrCmp95({'long_strings': False, 'external': True}),
 'person_245c': Jaro({'qval': 1, 'long_tolerance': False, 'winklerize': False, 'external': True}),
 'pu

The similarity metric decided for each attribute has been added as an additional kind of information to the columns metadata dictionary. The following table gives this summary in a structured form and lists the metric used for each attribute. Attributes with the same font color indicate similar types of values (description column) for better orientation.

| attribute     | subtype | description | similarity metric |
| ------------- |:--------|:------------|:------------------|
|<font color='red'>[coordinate](#coordinate)</font>|<font color='red'>\_E</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|               |<font color='red'>\_N</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|<font color='blue'>[corporate](#corporate)</font>|<font color='blue'>\_full</font>|<font color='blue'>Name</font>|<font color='blue'>LCSStr</font>|
|<font color='green'>[doi](#doi)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='orange'>[edition](#edition)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='black'>[exactDate](#exactDate)</font>|         |<font color='black'>Date</font>|<font color='black'>Hamming</font>|
|<font color='red'>[format](#format)</font>|<font color='red'>\_prefix</font>|<font color='red'>Code(2)</font>|<font color='red'>Identity</font>|
|               |<font color='red'>\_postfix</font>|<font color='red'>Code(6)</font>|<font color='red'>Jaccard (qval=2)</font>|
|<font color='green'>[isbn](#isbn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[ismn](#ismn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[musicid](#musicid)</font>|         |<font color='green'>Identifier</font>|<font color='green'>LCSStr</font>|
|<font color='orange'>[part](#part)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|
|<font color='blue'>[person](#person)</font>|<font color='blue'>\_100</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_700</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_245c</font>|<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='blue'>[pubinit](#pubinit)</font>|         |<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[scale](#scale)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='blue'>[ttlfull](#ttlfull)</font>|<font color='blue'>\_245</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|               |<font color='blue'>\_246</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[volumes](#volumes)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|

### Full Feature Matrix with Target Vector Handover

To hand over the resulting DataFrame of this chapter, the DataFrame is saved into a pickle file that will be read in the next chapters [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb) as input.

In [78]:
# Store into compressed intermediary file
with bz2.BZ2File(os.path.join(path_goldstandard,
                       'labelled_feature_matrix_full.pkl'), 'w') as df_output_file:
    pk.dump(df_attribute_with_sim_feature, df_output_file)
    
# Binary intermediary DataFrame file for docid's
with open(os.path.join(path_goldstandard, 'index_docids_df.pkl'), 'wb') as df_output_file:
    pk.dump(df_index_docids, df_output_file)

The full metadata dictionary is to be persisted for handover to subsequent chapters.

In [79]:
# The target is still needed for the feature matrix
columns_metadata_dict['features'].append('duplicates')

for k in columns_metadata_dict.keys():
    print(k, '\n', columns_metadata_dict[k], '\n')

data_analysis_columns 
 ['coordinate_E', 'coordinate_N', 'corporate_full', 'doi', 'edition', 'exactDate', 'format_prefix', 'format_postfix', 'isbn', 'ismn', 'musicid', 'part', 'person_100', 'person_700', 'person_245c', 'pubinit', 'scale', 'ttlfull_245', 'ttlfull_246', 'volumes'] 

columns_to_use 
 ['duplicates', 'coordinate_E_x', 'coordinate_E_y', 'coordinate_N_x', 'coordinate_N_y', 'corporate_full_x', 'corporate_full_y', 'doi_x', 'doi_y', 'edition_x', 'edition_y', 'exactDate_x', 'exactDate_y', 'format_prefix_x', 'format_prefix_y', 'format_postfix_x', 'format_postfix_y', 'isbn_x', 'isbn_y', 'ismn_x', 'ismn_y', 'musicid_x', 'musicid_y', 'part_x', 'part_y', 'person_100_x', 'person_100_y', 'person_700_x', 'person_700_y', 'person_245c_x', 'person_245c_y', 'pubinit_x', 'pubinit_y', 'scale_x', 'scale_y', 'ttlfull_245_x', 'ttlfull_245_y', 'ttlfull_246_x', 'ttlfull_246_y', 'volumes_x', 'volumes_y'] 

similarity_metrics 
 {'coordinate_E': LCSStr({'qval': 1, 'external': True}), 'coordinate_N': L

In [80]:
# Binary intermediary metadata file
with open(os.path.join(path_goldstandard,
                       'columns_metadata.pkl'), 'wb') as dict_output_file:
    pk.dump(columns_metadata_dict, dict_output_file)